In [1]:
import random
import time
import threading
import pygame
import sys
import os 

lane_counters = {'right': 0, 'down': 0, 'left': 0, 'up': 0}


# Default values of signal timers
defaultGreen = {0:10, 1:10, 2:10, 3:10}
defaultRed = 150
defaultYellow = 5

signals = []
noOfSignals = 4
currentGreen = 0   # Indicates which signal is green currently
nextGreen = (currentGreen+1)%noOfSignals    # Indicates which signal will turn green next
currentYellow = 0   # Indicates whether yellow signal is on or off 

speeds = {'car':2.25, 'bus':1.8, 'truck':1.8, 'bike':2.5}  # average speeds of vehicles

# Coordinates of vehicles' start
x = {'right':[0,0,0], 'down':[755,727,697], 'left':[1400,1400,1400], 'up':[602,627,657]}    
y = {'right':[348,370,398], 'down':[0,0,0], 'left':[498,466,436], 'up':[800,800,800]}

vehicles = {'right': {0:[], 1:[], 2:[], 'crossed':0}, 'down': {0:[], 1:[], 2:[], 'crossed':0}, 'left': {0:[], 1:[], 2:[], 'crossed':0}, 'up': {0:[], 1:[], 2:[], 'crossed':0}}
vehicleTypes = {0:'car', 1:'bus', 2:'truck', 3:'bike'}
directionNumbers = {0:'right', 1:'down', 2:'left', 3:'up'}

# Coordinates of signal image, timer, and vehicle count
signalCoods = [(530,230),(810,230),(810,570),(530,570)]
signalTimerCoods = [(530,210),(810,210),(810,550),(530,550)]

# Coordinates of stop lines
stopLines = {'right': 590, 'down': 330, 'left': 800, 'up': 535}
defaultStop = {'right': 580, 'down': 320, 'left': 810, 'up': 545}
# stops = {'right': [580,580,580], 'down': [320,320,320], 'left': [810,810,810], 'up': [545,545,545]}

# Gap between vehicles
stoppingGap = 15    # stopping gap
movingGap = 15   # moving gap

timeElapsed = 0
simulationTime = 300
timeElapsedCoods = (1100,50)

pygame.init()
simulation = pygame.sprite.Group()

class TrafficSignal:
    def __init__(self, red, yellow, green):
        self.red = red
        self.yellow = yellow
        self.green = green
        self.signalText = ""
        
class Vehicle(pygame.sprite.Sprite):
    def __init__(self, lane, vehicleClass, direction_number, direction):
        pygame.sprite.Sprite.__init__(self)
        self.lane = lane
        self.vehicleClass = vehicleClass
        self.speed = speeds[vehicleClass]
        self.direction_number = direction_number
        self.direction = direction
        self.x = x[direction][lane]
        self.y = y[direction][lane]
        self.crossed = 0
        vehicles[direction][lane].append(self)
        self.index = len(vehicles[direction][lane]) - 1
        path = "images/" + direction + "/" + vehicleClass + ".png"
        self.image = pygame.image.load(path)

        if(len(vehicles[direction][lane])>1 and vehicles[direction][lane][self.index-1].crossed==0):    # if more than 1 vehicle in the lane of vehicle before it has crossed stop line
            if(direction=='right'):
                self.stop = vehicles[direction][lane][self.index-1].stop - vehicles[direction][lane][self.index-1].image.get_rect().width - stoppingGap         # setting stop coordinate as: stop coordinate of next vehicle - width of next vehicle - gap
            elif(direction=='left'):
                self.stop = vehicles[direction][lane][self.index-1].stop + vehicles[direction][lane][self.index-1].image.get_rect().width + stoppingGap
            elif(direction=='down'):
                self.stop = vehicles[direction][lane][self.index-1].stop - vehicles[direction][lane][self.index-1].image.get_rect().height - stoppingGap
            elif(direction=='up'):
                self.stop = vehicles[direction][lane][self.index-1].stop + vehicles[direction][lane][self.index-1].image.get_rect().height + stoppingGap
        else:
            self.stop = defaultStop[direction]
            
            
            
        # Set new starting and stopping coordinate
        if(direction=='right'):
            temp = self.image.get_rect().width + stoppingGap    
            x[direction][lane] -= temp
        elif(direction=='left'):
            temp = self.image.get_rect().width + stoppingGap
            x[direction][lane] += temp
        elif(direction=='down'):
            temp = self.image.get_rect().height + stoppingGap
            y[direction][lane] -= temp
        elif(direction=='up'):
            temp = self.image.get_rect().height + stoppingGap
            y[direction][lane] += temp
        simulation.add(self)

    def render(self, screen):
        screen.blit(self.image, (self.x, self.y))

    def move(self):
        if(self.direction=='right'):
            if(self.crossed==0 and self.x+self.image.get_rect().width>stopLines[self.direction]):   # if the image has crossed stop line now
                self.crossed = 1
            if((self.x+self.image.get_rect().width<=self.stop or self.crossed == 1 or (currentGreen==0 and currentYellow==0)) and (self.index==0 or self.x+self.image.get_rect().width<(vehicles[self.direction][self.lane][self.index-1].x - movingGap))):                
            # (if the image has not reached its stop coordinate or has crossed stop line or has green signal) and (it is either the first vehicle in that lane or it is has enough gap to the next vehicle in that lane)
                self.x += self.speed  # move the vehicle
        elif(self.direction=='down'):
            if(self.crossed==0 and self.y+self.image.get_rect().height>stopLines[self.direction]):
                self.crossed = 1
            if((self.y+self.image.get_rect().height<=self.stop or self.crossed == 1 or (currentGreen==1 and currentYellow==0)) and (self.index==0 or self.y+self.image.get_rect().height<(vehicles[self.direction][self.lane][self.index-1].y - movingGap))):                
                self.y += self.speed
        elif(self.direction=='left'):
            if(self.crossed==0 and self.x<stopLines[self.direction]):
                self.crossed = 1
            if((self.x>=self.stop or self.crossed == 1 or (currentGreen==2 and currentYellow==0)) and (self.index==0 or self.x>(vehicles[self.direction][self.lane][self.index-1].x + vehicles[self.direction][self.lane][self.index-1].image.get_rect().width + movingGap))):                
                self.x -= self.speed   
        elif(self.direction=='up'):
            if(self.crossed==0 and self.y<stopLines[self.direction]):
                self.crossed = 1
            if((self.y>=self.stop or self.crossed == 1 or (currentGreen==3 and currentYellow==0)) and (self.index==0 or self.y>(vehicles[self.direction][self.lane][self.index-1].y + vehicles[self.direction][self.lane][self.index-1].image.get_rect().height + movingGap))):                
                self.y -= self.speed
        if self.crossed == 1:
            return  # Skip counting if the vehicle has already crossed

        if (
            (self.direction == 'right' and self.x + self.image.get_rect().width > stopLines[self.direction]) or
            (self.direction == 'down' and self.y + self.image.get_rect().height > stopLines[self.direction]) or
            (self.direction == 'left' and self.x < stopLines[self.direction]) or
            (self.direction == 'up' and self.y < stopLines[self.direction])
        ):
            self.crossed = 1
            lane_counters[self.direction] += 1

# Initialization of signals with default values
def initialize():
    ts1 = TrafficSignal(0, defaultYellow, defaultGreen[0])
    signals.append(ts1)
    ts2 = TrafficSignal(ts1.red+ts1.yellow+ts1.green, defaultYellow, defaultGreen[1])
    signals.append(ts2)
    ts3 = TrafficSignal(defaultRed, defaultYellow, defaultGreen[2])
    signals.append(ts3)
    ts4 = TrafficSignal(defaultRed, defaultYellow, defaultGreen[3])
    signals.append(ts4)
    repeat()

def repeat():
    global currentGreen, currentYellow, nextGreen
    while signals[currentGreen].red > 0:  # while the red timer of current green signal is not zero
        printStatus()
        updateValues()
        time.sleep(1)

    currentYellow = 1  # set yellow signal on

    while signals[currentGreen].yellow > 0:  # while the timer of current yellow signal is not zero
        printStatus()
        updateValues()
        time.sleep(1)

    currentYellow = 0  # set yellow signal off

    while signals[currentGreen].green > 0:  # while the timer of current green signal is not zero
        printStatus()
        updateValues()
        time.sleep(1)

    # Reset all signal times of current signal to default times
    signals[currentGreen].green = defaultGreen[currentGreen]
    signals[currentGreen].yellow = defaultYellow
    signals[currentGreen].red = defaultRed

    currentGreen = nextGreen  # set next signal as green signal
    nextGreen = (currentGreen + 1) % noOfSignals  # set next green signal
    signals[nextGreen].red = signals[currentGreen].yellow + signals[currentGreen].green  # set the red time of next to next signal as (yellow time + green time) of next signal

    repeat()


    
# Update values of the signal timers after every second
def updateValues():
    for i in range(0, noOfSignals):
        if(i==currentGreen):
            if(currentYellow==0):
                signals[i].green-=1
            else:
                signals[i].yellow-=1
        else:
            signals[i].red-=1

            
# Generating vehicles in the simulation
def generateVehicles():
    while(True):
        vehicle_type = random.randint(0,3)
        lane_number = random.randint(1,2)
        temp = random.randint(0,99)
        direction_number = 0
        dist = [25,50,75,100]
        if(temp<dist[0]):
            direction_number = 0
        elif(temp<dist[1]):
            direction_number = 1
        elif(temp<dist[2]):
            direction_number = 2
        elif(temp<dist[3]):
            direction_number = 3
        Vehicle(lane_number, vehicleTypes[vehicle_type], direction_number, directionNumbers[direction_number])
        time.sleep(1)
        
def printStatus():
    for i in range(0, 4):
        if(signals[i] != None):
            if(i==currentGreen):
                if(currentYellow==0):
                    print(" GREEN TS",i+1,"-> r:",signals[i].red," y:",signals[i].yellow," g:",signals[i].green)
                else:
                    print("YELLOW TS",i+1,"-> r:",signals[i].red," y:",signals[i].yellow," g:",signals[i].green)
            else:
                print("   RED TS",i+1,"-> r:",signals[i].red," y:",signals[i].yellow," g:",signals[i].green)
    print()
    
def showStats():
    totalVehicles = 0
    print('Direction-wise Vehicle Counts')
    for i in range(0,4):
        if lane_counters[directionNumbers[i]] != None:
            print(f'Direction {i + 1}: {lane_counters[directionNumbers[i]]}')
            totalVehicles += lane_counters[directionNumbers[i]]
    print('Total vehicles passed:', totalVehicles)
    print('Total simulation time:', timeElapsed)
    
def simTime():
    global timeElapsed, simulationTime
    while(True):
        timeElapsed += 1
        time.sleep(1)
        if(timeElapsed==simulationTime):
            showStats()
            os._exit(1)
            
            

class Main:
    thread1 = threading.Thread(name="initialization",target=initialize, args=())    # initialization
    thread1.daemon = True
    thread1.start()

    # Colours 
    black = (0, 0, 0)
    white = (255, 255, 255)

    # Screensize 
    screenWidth = 1400
    screenHeight = 800
    screenSize = (screenWidth, screenHeight)

    # Setting background image i.e. image of intersection
    background = pygame.image.load('intersection.png')

    screen = pygame.display.set_mode(screenSize)
    pygame.display.set_caption("SIMULATION")

    # Loading signal images and font
    redSignal = pygame.image.load('red.png')
    yellowSignal = pygame.image.load('yellow.png')
    greenSignal = pygame.image.load('green.png')
    font = pygame.font.Font(None, 30)

    thread2 = threading.Thread(name="generateVehicles",target=generateVehicles, args=())    # Generating vehicles
    thread2.daemon = True
    thread2.start()
    
    thread3 = threading.Thread(name="simTime",target=simTime, args=()) 
    thread3.daemon = True
    thread3.start()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                showStats()
                sys.exit()

        screen.blit(background,(0,0))   # display background in simulation
        for i in range(0,noOfSignals):  # display signal and set timer according to current status: green, yello, or red
            if(i==currentGreen):
                if(currentYellow==1):
                    signals[i].signalText = signals[i].yellow
                    screen.blit(yellowSignal, signalCoods[i])
                else:
                    signals[i].signalText = signals[i].green
                    screen.blit(greenSignal, signalCoods[i])
            else:
                if(signals[i].red<=10):
                    signals[i].signalText = signals[i].red
                else:
                    signals[i].signalText = "---"
                screen.blit(redSignal, signalCoods[i])
        signalTexts = ["","","",""]

        # display signal timer
        for i in range(0,noOfSignals):  
            signalTexts[i] = font.render(str(signals[i].signalText), True, white, black)
            screen.blit(signalTexts[i],signalTimerCoods[i])

        # display the vehicles
        for vehicle in simulation:  
            screen.blit(vehicle.image, [vehicle.x, vehicle.y])
            vehicle.move()
                # Display lane counters
        lane_text = [
            font.render(f"Right: {lane_counters['right']}", True, white, black),
            font.render(f"Down: {lane_counters['down']}", True, white, black),
            font.render(f"Left: {lane_counters['left']}", True, white, black),
            font.render(f"Up: {lane_counters['up']}", True, white, black),
        ]
        # Position the lane counters on the screen
        lane_counter_positions = [(20, 20), (20, 60), (20, 100), (20, 140)]
        # Display lane counters on the screen
        for i in range(noOfSignals):
            screen.blit(lane_text[i], lane_counter_positions[i])
            
        
        # display time elapsed
        timeElapsedText = font.render(("Time Elapsed: "+str(timeElapsed)), True, black, white)
        screen.blit(timeElapsedText,timeElapsedCoods)
            
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                showStats()  # Call the showStats() function when the user closes the simulation window
                sys.exit()

        
            
        pygame.display.update()


        
        
if __name__ == '__main__':
    Main()

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
YELLOW TS 1 -> r: 0  y: 5  g: 10
   RED TS 2 -> r: 15  y: 5  g: 10
   RED TS 3 -> r: 150  y: 5  g: 10
   RED TS 4 -> r: 150  y: 5  g: 10

YELLOW TS 1 -> r: 0  y: 4  g: 10
   RED TS 2 -> r: 14  y: 5  g: 10
   RED TS 3 -> r: 149  y: 5  g: 10
   RED TS 4 -> r: 149  y: 5  g: 10

YELLOW TS 1 -> r: 0  y: 3  g: 10
   RED TS 2 -> r: 13  y: 5  g: 10
   RED TS 3 -> r: 148  y: 5  g: 10
   RED TS 4 -> r: 148  y: 5  g: 10

YELLOW TS 1 -> r: 0  y: 2  g: 10
   RED TS 2 -> r: 12  y: 5  g: 10
   RED TS 3 -> r: 147  y: 5  g: 10
   RED TS 4 -> r: 147  y: 5  g: 10

YELLOW TS 1 -> r: 0  y: 1  g: 10
   RED TS 2 -> r: 11  y: 5  g: 10
   RED TS 3 -> r: 146  y: 5  g: 10
   RED TS 4 -> r: 146  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 10
   RED TS 2 -> r: 10  y: 5  g: 10
   RED TS 3 -> r: 145  y: 5  g: 10
   RED TS 4 -> r: 145  y: 5  g: 10

Direction-wise Vehicle Counts
Direction 1: 1
Direction 2: 0


SystemExit: 

C:\Users\yicho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


 GREEN TS 1 -> r: 0  y: 0  g: 9
   RED TS 2 -> r: 9  y: 5  g: 10
   RED TS 3 -> r: 144  y: 5  g: 10
   RED TS 4 -> r: 144  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 8
   RED TS 2 -> r: 8  y: 5  g: 10
   RED TS 3 -> r: 143  y: 5  g: 10
   RED TS 4 -> r: 143  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 7
   RED TS 2 -> r: 7  y: 5  g: 10
   RED TS 3 -> r: 142  y: 5  g: 10
   RED TS 4 -> r: 142  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 6
   RED TS 2 -> r: 6  y: 5  g: 10
   RED TS 3 -> r: 141  y: 5  g: 10
   RED TS 4 -> r: 141  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 5
   RED TS 2 -> r: 5  y: 5  g: 10
   RED TS 3 -> r: 140  y: 5  g: 10
   RED TS 4 -> r: 140  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 4
   RED TS 2 -> r: 4  y: 5  g: 10
   RED TS 3 -> r: 139  y: 5  g: 10
   RED TS 4 -> r: 139  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 3
   RED TS 2 -> r: 3  y: 5  g: 10
   RED TS 3 -> r: 138  y: 5  g: 10
   RED TS 4 -> r: 138  y: 5  g: 10

 GREEN TS 1 -> r: 0  y: 0  g: 2
   RED TS 2 -> r